# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_after.png" alt="Image 2" style="width:300px; border:1px solid #ccc; border-radius:6px;">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

First, let's connect to our SQLite database containing music store data.


In [ ]:
import { SqlDatabase } from "@langchain/classic/sql_db";
import { DataSource } from "typeorm";

const datasource = new DataSource({
    type: "sqlite",
    database: "./Chinook.db", // Replace with the link to your database
});
const db = await SqlDatabase.fromDataSourceParams({
    appDataSource: datasource,
});

Define a context schema to hold customer information (first and last name) that will be available throughout the agent's execution.


In [ ]:
import { z } from "zod";

const contextSchema = z.object({
    db: z.instanceof(SqlDatabase)
});
type Context = z.infer<typeof contextSchema>;

Create a tool to execute SQL queries. It supports named parameters (`:first`, `:last`) that get replaced with values from the runtime context.


In [ ]:
import { tool, Runtime } from "langchain";

const executeSQL = tool(async ({ query }, runtime: Runtime<Context>) => {
    return await db.run(query);
}, {
    name: "execute_sql",
    description: "Execute a SQLite command and return results.",
    schema: z.object({
        query: z.string()
    })
});

Define the system prompt that instructs the agent how to interact with the database safely and use named parameters.


In [ ]:
export const SYSTEM = `You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool \`execute_sql\` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.`

Create the agent with our tools and system prompt. Note: **no checkpointer yet**, so the agent won't remember previous conversations.


In [ ]:
import * as setup from "./setup.ts";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "anthropic:claude-sonnet-4-5-20250929",
    tools: [executeSQL],
    systemPrompt: SYSTEM,
    contextSchema,
})

## Repeated Queries

Ask about Frank Harris's last invoice. The agent successfully retrieves the information.


In [ ]:
const stream = await agent.stream({
    messages: "This is Frank Harris, What was the total on my last invoice?",
}, {
    streamMode: "values",
    context: { db }
})

for await (const step of stream) {
    displayMessage(step.messages.at(-1))
}

Now ask a follow-up question: "What were the titles?" Without memory, the agent has no idea what we're referring to and asks for clarification.


In [ ]:
const stream = await agent.stream({
    messages: "What were the titles?",
}, {
    streamMode: "values",
    context: { db }
})

for await (const step of stream) {
    displayMessage(step.messages.at(-1))
}

## Add memory

Let's fix this by adding a checkpointer. The `MemorySaver` will store conversation history so the agent can remember context across turns.


In [ ]:
import * as setup from "./setup.ts";
import { createAgent } from "langchain";
import { MemorySaver } from "@langchain/langgraph";

const checkpointer = new MemorySaver()
const agent = createAgent({
    model: "anthropic:claude-sonnet-4-5-20250929",
    tools: [executeSQL],
    systemPrompt: SYSTEM,
    contextSchema,
    checkpointer
})

Ask the same question again, but this time with a `thread_id` to track the conversation.


In [ ]:
const stream = await agent.stream({
    messages: "This is Frank Harris, What was the total on my last invoice?",
}, {
    streamMode: "values",
    configurable: { thread_id: "1" },
    context: { db }
})

for await (const step of stream) {
    displayMessage(step.messages.at(-1))
}

Now when we ask "What were the titles?", the agent remembers the previous question about the invoice and returns the track titles! 🎵


In [ ]:
const stream = await agent.stream({
    messages: "What were the titles?",
}, {
    streamMode: "values",
    configurable: { thread_id: "1" },
    context: { db }
})

for await (const step of stream) {
    displayMessage(step.messages.at(-1))
}